In [8]:
import pyspark as pyspark
from pyspark.sql import SparkSession

In [36]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [10]:
spark = SparkSession.builder.appName("Maternal Mortality Data Processing").config("spark.memory.offHeap.enabled","true").config("spark.memory.offHeap.size","10g").getOrCreate()

In [28]:
df = spark.read.csv('VSRR_Provisional_Maternal_Death_Counts_and_Rates.csv', header=True, escape="\"")

In [13]:
df.show(5,0)

+----------+-------------+-----+--------+-------------+--------------+---------------+-----------------+---------------+-----------+-----------------------+--------+
|Data As Of|Jurisdiction |Group|Subgroup|Year of Death|Month of Death|Time Period    |Month Ending Date|Maternal Deaths|Live Births|Maternal Mortality Rate|Footnote|
+----------+-------------+-----+--------+-------------+--------------+---------------+-----------------+---------------+-----------+-----------------------+--------+
|4/14/2024 |United States|Total|Total   |2019         |1             |12 month-ending|1/31/2019        |660            |3,787,776  |17.4                   |NULL    |
|4/14/2024 |United States|Total|Total   |2019         |2             |12 month-ending|2/28/2019        |653            |3,783,489  |17.3                   |NULL    |
|4/14/2024 |United States|Total|Total   |2019         |3             |12 month-ending|3/31/2019        |657            |3,771,682  |17.4                   |NULL    |
|4/1

In [29]:
### Take a look at the columns
df.columns

['Data As Of',
 'Jurisdiction',
 'Group',
 'Subgroup',
 'Year of Death',
 'Month of Death',
 'Time Period',
 'Month Ending Date',
 'Maternal Deaths',
 'Live Births',
 'Maternal Mortality Rate',
 'Footnote']

In [30]:
### Get rid of the unnecessary columns
### 1) 'Data As Of'
df.select("Data As Of").show() 
## It looks like all the values are the same, so let's delete this column
df = df.drop("Data As Of")
df.columns

+----------+
|Data As Of|
+----------+
| 4/14/2024|
| 4/14/2024|
| 4/14/2024|
| 4/14/2024|
| 4/14/2024|
| 4/14/2024|
| 4/14/2024|
| 4/14/2024|
| 4/14/2024|
| 4/14/2024|
| 4/14/2024|
| 4/14/2024|
| 4/14/2024|
| 4/14/2024|
| 4/14/2024|
| 4/14/2024|
| 4/14/2024|
| 4/14/2024|
| 4/14/2024|
| 4/14/2024|
+----------+
only showing top 20 rows



['Jurisdiction',
 'Group',
 'Subgroup',
 'Year of Death',
 'Month of Death',
 'Time Period',
 'Month Ending Date',
 'Maternal Deaths',
 'Live Births',
 'Maternal Mortality Rate',
 'Footnote']

In [31]:
df.show(5,0)

+-------------+-----+--------+-------------+--------------+---------------+-----------------+---------------+-----------+-----------------------+--------+
|Jurisdiction |Group|Subgroup|Year of Death|Month of Death|Time Period    |Month Ending Date|Maternal Deaths|Live Births|Maternal Mortality Rate|Footnote|
+-------------+-----+--------+-------------+--------------+---------------+-----------------+---------------+-----------+-----------------------+--------+
|United States|Total|Total   |2019         |1             |12 month-ending|1/31/2019        |660            |3,787,776  |17.4                   |NULL    |
|United States|Total|Total   |2019         |2             |12 month-ending|2/28/2019        |653            |3,783,489  |17.3                   |NULL    |
|United States|Total|Total   |2019         |3             |12 month-ending|3/31/2019        |657            |3,771,682  |17.4                   |NULL    |
|United States|Total|Total   |2019         |4             |12 month-en

In [39]:
### Jurisdiction column
##df.select("Jurisdiction").show() ##UNCOMMENT LATER?
### It looks like all the values under the 'Jursidiction' column are the same, but let's double-check
df_juris = df.groupBy('Jurisdiction').agg(count('Jurisdiction'))
df_juris.show(5,0)
### We see that the 'Jurisdiction' column only consists of entries reading 'United States', so let's delete that column as well
df = df.drop('Jurisdiction')
df.show(5,0)

+-------------+-------------------+
|Jurisdiction |count(Jurisdiction)|
+-------------+-------------------+
|United States|600                |
+-------------+-------------------+

+-----+--------+-------------+--------------+---------------+-----------------+---------------+-----------+-----------------------+--------+
|Group|Subgroup|Year of Death|Month of Death|Time Period    |Month Ending Date|Maternal Deaths|Live Births|Maternal Mortality Rate|Footnote|
+-----+--------+-------------+--------------+---------------+-----------------+---------------+-----------+-----------------------+--------+
|Total|Total   |2019         |1             |12 month-ending|1/31/2019        |660            |3,787,776  |17.4                   |NULL    |
|Total|Total   |2019         |2             |12 month-ending|2/28/2019        |653            |3,783,489  |17.3                   |NULL    |
|Total|Total   |2019         |3             |12 month-ending|3/31/2019        |657            |3,771,682  |17.4   